In [1]:
import sqlalchemy
import pandas as pd
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import inspect
import openpyxl
%reload_ext autoreload
%autoreload 2
from geolocate import geolocate_batch

In [2]:
server = 'localhost'
database = 't_obras'
username = 'root'
password = input("Digite a senha do mysql: ")
driver = 'mysql+pymysql'
connection_string = f'{driver}://{username}:{password}@{server}/{database}'
engine = sqlalchemy.create_engine(connection_string)

In [3]:
orgaos = pd.read_sql_table('work_nome_orgaos', con=engine)
orgaos.head()

,CD_OBRA,ORGAO_CONTRATANTE,ORGAO_EXECUTOR,NR_PROCESSO,NM_FAVORECIDO,CNPJ
0,1,GEO-RIO - Fundação Instituto de Geotécnica do ...,GEO-RIO - Fundação Instituto de Geotécnica do ...,0006/101231/2011,CONCREMAT ENGENHARIA E TECNOLOGIA S.A.,33.146.648/0001-20
1,2,GEO-RIO - Fundação Instituto de Geotécnica do ...,GEO-RIO - Fundação Instituto de Geotécnica do ...,0006/101244/2011,SOLOTESTE ENGENHARIA LTDA,33.456.492/0001-83
2,3,GEO-RIO - Fundação Instituto de Geotécnica do ...,GEO-RIO - Fundação Instituto de Geotécnica do ...,0006/101243/2011,RESITEC DE TERESOPOLIS CONSTRUCOES LTDA.,03.253.042/0001-22
3,4,GEO-RIO - Fundação Instituto de Geotécnica do ...,GEO-RIO - Fundação Instituto de Geotécnica do ...,0006/101242/2011,SEEL SERVICOS ESPECIAIS DE ENGENHARIA LTDA,72.030.927/0001-85
4,5,GEO-RIO - Fundação Instituto de Geotécnica do ...,GEO-RIO - Fundação Instituto de Geotécnica do ...,0006/101248/2011,J.S.32 CONSTRUCOES LTDA,09.466.677/0001-94


In [4]:
excel2 = r'G:\Outros computadores\acer\gits\transparencia_obras_reformulado\work_data\orgaos.xlsx'
nome_sigla = pd.read_excel(excel2, sheet_name='orgaos')
nome_sigla.head()

,Sigla,Nome
0,AC,Secretaria Municipal de Ação Comunitária
1,CVL,Secretaria Municipal da Casa Civil
2,F,Secretaria Municipal de Fazenda
3,FPJ,Fundação Parques e Jardins
4,GEO-RIO,Fundação Instituto de Geotécnica do Município ...


In [5]:
# unir coluna NM_FAVORECIDO com coluna CNPJ
orgaos['NM_FAVORECIDO'] = orgaos['NM_FAVORECIDO'] + ' - ' + orgaos['CNPJ']

In [6]:
# apagar coluna CNPJ
orgaos.drop('CNPJ', axis=1, inplace=True)

In [7]:
# duplicar coluna orgao_contratante
orgaos['ORGAO_CONTRATANTE_2'] = orgaos['ORGAO_CONTRATANTE']
# extrair texto antes da / da coluina orgao_contratante
orgaos['ORGAO_CONTRATANTE'] = orgaos['ORGAO_CONTRATANTE'].str.split('/').str[0]

In [8]:
# duplicar coluna orgao_executor
orgaos['ORGAO_EXECUTOR_2'] = orgaos['ORGAO_EXECUTOR']
# extrair texto antes da / da coluina orgao_executor
orgaos['ORGAO_EXECUTOR'] = orgaos['ORGAO_EXECUTOR'].str.split('/').str[0]

In [10]:
# RENOMEAR COLUNA ORGAO_CONTRATANTE
orgaos.rename(columns={'ORGAO_CONTRATANTE': 'contratante_primeiro_nivel'}, inplace=True)
# RENOMEAR COLUNA ORGAO_EXECUTOR
orgaos.rename(columns={'ORGAO_EXECUTOR': 'executor_primeiro_nivel'}, inplace=True)

In [11]:
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('RIO-AGUAS', 'RIO-ÁGUAS')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('RIO-AGUAS', 'RIO-ÁGUAS')

In [12]:
# substituir valores da coluna contratante_primeiro_nivel
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('AC', 'AC - Secretaria Municipal de Ação Comunitária')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('CVL', 'CVL - Secretaria Municipal da Casa Civil')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('F', ' F - Secretaria Municipal de Fazenda')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('FPJ', 'FPJ - Fundação Parques e Jardins')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('GEO-RIO', 'GEO-RIO - Fundação Instituto de Geotécnica do Município do Rio de Janeiro')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('H', 'H - Secretaria Municipal de Habitação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('I', 'I - Secretaria Municipal de Infraestrutura')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('IH', 'IH - Secretaria Municipal de Infraestrutura e Habitação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('IHC', 'IHC - Secretaria Municipal de Infraestrutura, Habitação e Conservação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('O', 'O - Secretaria Municipal de Obras')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('PREVI-RIO', 'PREVI-RIO - Instituto de Previdência e Assistência')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('QV', 'QV - Secretaria Municipal do Envelhecimento Saudável e Qualidade de Vida')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('RIO-ÁGUAS', 'RIO-ÁGUAS - Fundação Instituto das Águas do Município do Rio de Janeiro')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('RIOLUZ', 'RIOLUZ - Companhia Municipal de Iluminação Pública')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('RIO-URBE', 'RIO-URBE - Empresa Municipal de Urbanização')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SC', 'SC - Secretaria Municipal de Conservação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SCMA', 'SCMA - Secretaria Municipal de Conservação e Meio Ambiente')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SECONSERVA', 'SECONSERVA - Secretaria Municipal de Conservação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SEMESQV', 'SEMESQV - Secretaria Municipal do Envelhecimento Saudável e Qualidade de Vida')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMA', 'SMA - Secretaria Municipal de Administração ')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMA', 'SMA - Secretaria Municipal de Fazenda')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMH', 'SMH - Secretaria Municipal de Habitação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMHC', 'SMHC - Secretaria Municipal de Habitação e Cidadania')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMIH', 'SMIH - Secretaria Municipal de Infraestrutura e Habitação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMIHC', 'Secretaria Municipal de Infraestrutura, Habitação e Conservação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMO', 'SMIHC - Secretaria Municipal de Obras')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMUIH', 'SMUIH - Secretaria Municipal de Urbanismo, Infraestrutura e Habitação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('UIH', 'UIH - Secretaria Municipal de Urbanismo, Infraestrutura e Habitação')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('SMAR', 'SMAR - Secretaria Municipal de Saneamento e Recursos Hídricos')
orgaos['contratante_primeiro_nivel'] = orgaos['contratante_primeiro_nivel'].replace('MULTIRIO', 'MULTIRIO - Empresa Municipal de Multimeios Ltda')

In [13]:
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('AC', 'AC - Secretaria Municipal de Ação Comunitária')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('CVL', 'CVL - Secretaria Municipal da Casa Civil')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('F', 'F - Secretaria Municipal de Fazenda')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('FPJ', 'FPJ - Fundação Parques e Jardins')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('GEO-RIO', 'GEO-RIO - Fundação Instituto de Geotécnica do Município do Rio de Janeiro')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('H', 'H - Secretaria Municipal de Habitação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('I', 'I - Secretaria Municipal de Infraestrutura')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('IH', 'IH - Secretaria Municipal de Infraestrutura e Habitação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('IHC', 'IHC - Secretaria Municipal de Infraestrutura, Habitação e Conservação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('O', 'O - Secretaria Municipal de Obras')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('PREVI-RIO', 'PREVI-RIO - Instituto de Previdência e Assistência')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('QV', 'QV - Secretaria Municipal do Envelhecimento Saudável e Qualidade de Vida')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('RIO-ÁGUAS', 'RIO-ÁGUAS - Fundação Instituto das Águas do Município do Rio de Janeiro')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('RIOLUZ', 'RIOLUZ - Companhia Municipal de Iluminação Pública')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('RIO-URBE', 'RIO-URBE - Empresa Municipal de Urbanização')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SC', 'SC - Secretaria Municipal de Conservação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SCMA', 'SCMA - Secretaria Municipal de Conservação e Meio Ambiente')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SECONSERVA', 'SECONSERVA - Secretaria Municipal de Conservação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SEMESQV', 'SEMESQV - Secretaria Municipal do Envelhecimento Saudável e Qualidade de Vida')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMA', 'SMA - Secretaria Municipal de Administração ')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMF', 'SMF - Secretaria Municipal de Fazenda')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMH', 'SMH - Secretaria Municipal de Habitação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMHC', 'SMHC - Secretaria Municipal de Habitação e Cidadania')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMIH', 'SMIH - Secretaria Municipal de Infraestrutura e Habitação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMIHC', 'SMIHC - Secretaria Municipal de Infraestrutura, Habitação e Conservação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMO', 'SMO - Secretaria Municipal de Obras')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMUIH', 'SMUIH - Secretaria Municipal de Urbanismo, Infraestrutura e Habitação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('UIH', 'UIH - Secretaria Municipal de Urbanismo, Infraestrutura e Habitação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SEAC-RIO', 'SEAC-RIO - Sindicato das Empresas de Asseio e Conservação do Rio de Janeiro')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('CASA CIVIL', 'CASA CIVIL - Secretaria Municipal da Casa Civil')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SME', 'SME - Secretaria Municipal de Educação')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMI', 'SMI - Secretaria Municipal de Infraestrutura')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMS', 'SMS - Secretaria Municipal de Saúde')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMC', 'SMC - Secretaria Municipal de Cultura')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMU', 'SMU - Secretaria Municipal de Urbanismo')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMFP', 'SMFP - Secretaria Municipal de Fazenda e Planejamento')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SECONSERMA', 'SECONSERMA - Secretaria Municipal de Conservação e Meio Ambiente')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('SMAR', 'SMAR - Secretaria Municipal de Saneamento e Recursos Hídricos')
orgaos['executor_primeiro_nivel'] = orgaos['executor_primeiro_nivel'].replace('MULTIRIO', 'MULTIRIO - Empresa Municipal de Multimeios Ltda')

In [14]:
# atulizar o banco de dados
orgaos.to_sql('work_nome_orgaos', engine, if_exists='replace', index=False)